In [1]:
import requests
import pandas as pd
import lxml.etree as ET
from bs4 import BeautifulSoup as soup
import unicodedata
import tqdm
from time import sleep
import time

In [2]:
df = pd.read_hdf("prep_df.h5", key="df")
df

,idn,title,date,lang,urn,title_clean
0,1317002237,ängste der allgemeinbevölkerung in zeiten der ...,2024,ger,urn:nbn:de:bsz:14-qucosa2-890970,ängste allgemeinbevölkerung zeiten covid-19-pa...
1,131701071X,auswirkungen der covid-19-pandemie auf die pri...,2024,ger,urn:nbn:de:bsz:16-heidok-342525,auswirkungen covid-19-pandemie primärversorgun...
2,1319384919,auswirkungen der sars-cov-2-pandemie auf die b...,2024,ger,urn:nbn:de:bvb:19-331095,auswirkungen sars-cov-2-pandemie bedürfnisse p...
3,1319553818,beeinträchtigung des gesichtsfeldes durch das ...,2024,ger,urn:nbn:de:101:1-2024021604063338639840,beeinträchtigung gesichtsfeldes tragen mund-na...
4,131589002X,der serotonin-wiederaufnahmehemmer fluoxetin...,2024,ger,urn:nbn:de:bvb:20-opus-347190,der serotonin-wiederaufnahmehemmer fluoxetin...
...,...,...,...,...,...,...
2890,1275808069,zur rolle von unterhaltungsmagazinen in der pa...,2022,ger,urn:nbn:de:101:1-2022121621092079245245,rolle unterhaltungsmagazinen pandemie entdiffe...
2891,1308344058,zur seelischen gesundheit von studierenden – e...,2023,ger,urn:nbn:de:101:1-2023110209541661089956,seelischen gesundheit studierenden – bestandsa...
2892,1276936842,"zusammenhänge zwischen sozialer herkunft, unte...",2022,ger,urn:nbn:de:101:1-2022123109195820053242,zusammenhänge sozialer herkunft unterrichtsfor...
2893,1280215933,zuteilung knapper intensivkapazitäten in der p...,2022,ger,urn:nbn:de:101:1-2023020621022586089746,zuteilung knapper intensivkapazitäten pandemie...


# Vorbereitungen:

## Laden spezifischer Bibliotheken: 

In [3]:
import spacy
from spacy import displacy
import os
import plotly.graph_objects as go
import plotly.express as px

In [4]:
#Code für den Download eines der Sprachmodelle: 
#!python -m spacy download de

In [5]:
nlp = spacy.load('de_core_news_sm') ## Laden eines Trained Model for German

In [6]:
print(nlp.pipe_names)

['tok2vec', 'tagger', 'morphologizer', 'parser', 'lemmatizer', 'attribute_ruler', 'ner']


## Tokenization: 

In [7]:
def process_text(text):
    return nlp(text)

In [8]:
df['token'] = df['title_clean'].apply(process_text)
df

,idn,title,date,lang,urn,title_clean,token
0,1317002237,ängste der allgemeinbevölkerung in zeiten der ...,2024,ger,urn:nbn:de:bsz:14-qucosa2-890970,ängste allgemeinbevölkerung zeiten covid-19-pa...,"(ängste, allgemeinbevölkerung, zeiten, covid-1..."
1,131701071X,auswirkungen der covid-19-pandemie auf die pri...,2024,ger,urn:nbn:de:bsz:16-heidok-342525,auswirkungen covid-19-pandemie primärversorgun...,"(auswirkungen, covid-19-pandemie, primärversor..."
2,1319384919,auswirkungen der sars-cov-2-pandemie auf die b...,2024,ger,urn:nbn:de:bvb:19-331095,auswirkungen sars-cov-2-pandemie bedürfnisse p...,"(auswirkungen, sars-cov-2-pandemie, bedürfniss..."
3,1319553818,beeinträchtigung des gesichtsfeldes durch das ...,2024,ger,urn:nbn:de:101:1-2024021604063338639840,beeinträchtigung gesichtsfeldes tragen mund-na...,"(beeinträchtigung, gesichtsfeldes, tragen, mun..."
4,131589002X,der serotonin-wiederaufnahmehemmer fluoxetin...,2024,ger,urn:nbn:de:bvb:20-opus-347190,der serotonin-wiederaufnahmehemmer fluoxetin...,"(der, serotonin-wiederaufnahmehemmer, fluoxe..."
...,...,...,...,...,...,...,...
2890,1275808069,zur rolle von unterhaltungsmagazinen in der pa...,2022,ger,urn:nbn:de:101:1-2022121621092079245245,rolle unterhaltungsmagazinen pandemie entdiffe...,"(rolle, unterhaltungsmagazinen, pandemie, entd..."
2891,1308344058,zur seelischen gesundheit von studierenden – e...,2023,ger,urn:nbn:de:101:1-2023110209541661089956,seelischen gesundheit studierenden – bestandsa...,"(seelischen, gesundheit, studierenden, –, best..."
2892,1276936842,"zusammenhänge zwischen sozialer herkunft, unte...",2022,ger,urn:nbn:de:101:1-2022123109195820053242,zusammenhänge sozialer herkunft unterrichtsfor...,"(zusammenhänge, sozialer, herkunft, unterricht..."
2893,1280215933,zuteilung knapper intensivkapazitäten in der p...,2022,ger,urn:nbn:de:101:1-2023020621022586089746,zuteilung knapper intensivkapazitäten pandemie...,"(zuteilung, knapper, intensivkapazitäten, pand..."


## Lemmatization: 

In [9]:
def get_lemma(doc):
    return [(token.lemma_) for token in doc]

In [10]:
df['lemma'] = df['token'].apply(get_lemma)
df

,idn,title,date,lang,urn,title_clean,token,lemma
0,1317002237,ängste der allgemeinbevölkerung in zeiten der ...,2024,ger,urn:nbn:de:bsz:14-qucosa2-890970,ängste allgemeinbevölkerung zeiten covid-19-pa...,"(ängste, allgemeinbevölkerung, zeiten, covid-1...","[Angst, Allgemeinbevölkerung, Zeit, covid-19-p..."
1,131701071X,auswirkungen der covid-19-pandemie auf die pri...,2024,ger,urn:nbn:de:bsz:16-heidok-342525,auswirkungen covid-19-pandemie primärversorgun...,"(auswirkungen, covid-19-pandemie, primärversor...","[auswirkung, covid-19-pandemie, Primärversorgu..."
2,1319384919,auswirkungen der sars-cov-2-pandemie auf die b...,2024,ger,urn:nbn:de:bvb:19-331095,auswirkungen sars-cov-2-pandemie bedürfnisse p...,"(auswirkungen, sars-cov-2-pandemie, bedürfniss...","[auswirkung, sars-cov-2-pandemie, bedürfnisse,..."
3,1319553818,beeinträchtigung des gesichtsfeldes durch das ...,2024,ger,urn:nbn:de:101:1-2024021604063338639840,beeinträchtigung gesichtsfeldes tragen mund-na...,"(beeinträchtigung, gesichtsfeldes, tragen, mun...","[Beeinträchtigung, gesichtsfeld, tragen, mund-..."
4,131589002X,der serotonin-wiederaufnahmehemmer fluoxetin...,2024,ger,urn:nbn:de:bvb:20-opus-347190,der serotonin-wiederaufnahmehemmer fluoxetin...,"(der, serotonin-wiederaufnahmehemmer, fluoxe...","[der, serotonin-wiederaufnahmehemmer, Fluoxe..."
...,...,...,...,...,...,...,...,...
2890,1275808069,zur rolle von unterhaltungsmagazinen in der pa...,2022,ger,urn:nbn:de:101:1-2022121621092079245245,rolle unterhaltungsmagazinen pandemie entdiffe...,"(rolle, unterhaltungsmagazinen, pandemie, entd...","[Rolle, unterhaltungsmagazinen, Pandemie, entd..."
2891,1308344058,zur seelischen gesundheit von studierenden – e...,2023,ger,urn:nbn:de:101:1-2023110209541661089956,seelischen gesundheit studierenden – bestandsa...,"(seelischen, gesundheit, studierenden, –, best...","[seelisch, Gesundheit, Studierend, --, Bestand..."
2892,1276936842,"zusammenhänge zwischen sozialer herkunft, unte...",2022,ger,urn:nbn:de:101:1-2022123109195820053242,zusammenhänge sozialer herkunft unterrichtsfor...,"(zusammenhänge, sozialer, herkunft, unterricht...","[Zusammenhäng, sozial, Herkunft, unterrichtsfo..."
2893,1280215933,zuteilung knapper intensivkapazitäten in der p...,2022,ger,urn:nbn:de:101:1-2023020621022586089746,zuteilung knapper intensivkapazitäten pandemie...,"(zuteilung, knapper, intensivkapazitäten, pand...","[Zuteilung, knapp, intensivkapazität, Pandemie..."


### Einfache Worthäufigkeitsprüfung: 

In [11]:
# Hier zu suchenden Begriff eingeben: 
term = "corona"  

In [12]:
print(f'"{term}" appears in the text tokens column ' + str(df.token.apply(lambda x: str(x)).apply(lambda x: x.count(term)).sum()) + ' times.')#
print(f'"{term}" appears in the lemmas column ' + str(df['lemma'].apply(lambda x: str(x)).apply(lambda x: x.count(term)).sum()) + ' times.')

"corona" appears in the text tokens column 878 times.
"corona" appears in the lemmas column 675 times.


## Optional: Part of speech tags

In [13]:
def get_pos(doc):
    return [(token.pos_, token.tag_) for token in doc]

In [14]:
df['POS'] = df['token'].apply(get_pos)
df.head()

,idn,title,date,lang,urn,title_clean,token,lemma,POS
0,1317002237,ängste der allgemeinbevölkerung in zeiten der ...,2024,ger,urn:nbn:de:bsz:14-qucosa2-890970,ängste allgemeinbevölkerung zeiten covid-19-pa...,"(ängste, allgemeinbevölkerung, zeiten, covid-1...","[Angst, Allgemeinbevölkerung, Zeit, covid-19-p...","[(ADJ, ADJA), (NOUN, NN), (NOUN, NN), (PROPN, ..."
1,131701071X,auswirkungen der covid-19-pandemie auf die pri...,2024,ger,urn:nbn:de:bsz:16-heidok-342525,auswirkungen covid-19-pandemie primärversorgun...,"(auswirkungen, covid-19-pandemie, primärversor...","[auswirkung, covid-19-pandemie, Primärversorgu...","[(ADJ, ADJA), (ADJ, FM), (NOUN, NN), (ADJ, ADJ..."
2,1319384919,auswirkungen der sars-cov-2-pandemie auf die b...,2024,ger,urn:nbn:de:bvb:19-331095,auswirkungen sars-cov-2-pandemie bedürfnisse p...,"(auswirkungen, sars-cov-2-pandemie, bedürfniss...","[auswirkung, sars-cov-2-pandemie, bedürfnisse,...","[(ADJ, ADJA), (NOUN, NN), (ADJ, ADJA), (ADJ, A..."
3,1319553818,beeinträchtigung des gesichtsfeldes durch das ...,2024,ger,urn:nbn:de:101:1-2024021604063338639840,beeinträchtigung gesichtsfeldes tragen mund-na...,"(beeinträchtigung, gesichtsfeldes, tragen, mun...","[Beeinträchtigung, gesichtsfeld, tragen, mund-...","[(NOUN, NN), (ADJ, ADJA), (VERB, VVFIN), (X, N..."
4,131589002X,der serotonin-wiederaufnahmehemmer fluoxetin...,2024,ger,urn:nbn:de:bvb:20-opus-347190,der serotonin-wiederaufnahmehemmer fluoxetin...,"(der, serotonin-wiederaufnahmehemmer, fluoxe...","[der, serotonin-wiederaufnahmehemmer, Fluoxe...","[(PROPN, NE), (ADJ, ADJA), (NOUN, NN), (VERB, ..."


# Topic Modeling:

### Latent Dirichlet Allocation (LDA):¶

In [34]:
import gensim
from gensim import corpora
from gensim.parsing.preprocessing import preprocess_string

In [24]:
lemmatized_text = ' '.join(df['lemma'].explode())

#Alternative:
#lemmatized_text = df.token.apply(lambda x: ' '.join([token.lemma_ for token in nlp(x)]))

In [26]:
print(lemmatized_text)

Angst Allgemeinbevölkerung Zeit covid-19-pandemie der gesundheitlich bewältigung auswirkung covid-19-pandemie Primärversorgung patientinn Akut zerebrovaskulären Erkrankung auswirkung sars-cov-2-pandemie bedürfnisse patientinn Palliativversorgung sars-cov-2-diagnose der angehörig qualitativ Interviewstudie Beeinträchtigung gesichtsfeld tragen mund-nasen-bedeckung covid-19-pandemie der serotonin-wiederaufnahmehemmer Fluoxetin inhibieren Sars-cov-2-replikation Evaluation psychisch Belastungserlebens Gesundheitspersonal sars-cov-2 Pandemie -- Beurteilung anhand umfrageergebnissen anästhesiologisch Abteilung Universitätsklinikum Würzburg forensic Psychiatric implications post-covid -- long-covid = forensisch-psychiatrisch implikation post‑/long-covid Neuregelung pandemiebedingt Triage häufigkeit schwer diabetisch ketoazidos Kind jugendlichen neu diagnostiziert Bzw bekannt typ-1- typ-2- diabet Mellitus erster lockdowns covid-19 Pandemie 2020 Vergleich Zeitraum Jahr 2019 Universitätskinderk

In [28]:
#Erstellen eines Dictionary und einer Doc-Term-Matrix:
processed_text = preprocess_string(lemmatized_text)
dictionary = corpora.Dictionary([processed_text])
doc_term_matrix = [dictionary.doc2bow(doc.split()) for doc in processed_text]

In [31]:
#Alternative: 
#dictionary = corpora.Dictionary(lemmatized_text.apply(lambda x: x.split()))
#doc_term_matrix = [dictionary.doc2bow(doc.split()) for doc in lemmatized_text]


In [32]:
# Apply LDA for topic modeling
lda_model = gensim.models.LdaModel(doc_term_matrix, num_topics=3, id2word=dictionary, passes=15)

In [33]:
# Visualize the topics
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, doc_term_matrix, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.105709  0.204721       1        1  33.870414
2     -0.240136 -0.015696       2        1  33.089084
1      0.134427 -0.189025       3        1  33.040502, topic_info=              Term        Freq       Total Category  logprob  loglift
4013       pandemi  906.000000  906.000000  Default  30.0000  30.0000
1050        corona  561.000000  561.000000  Default  29.0000  29.0000
1086         covid  503.000000  503.000000  Default  28.0000  28.0000
6217          zeit  163.000000  163.000000  Default  27.0000  27.0000
3111         krise  143.000000  143.000000  Default  26.0000  26.0000
...            ...         ...         ...      ...      ...      ...
4441        rahmen   22.534377   23.039935   Topic3  -5.6717   1.0852
5520        umgang   19.226632   19.730425   Topic3  -5.8305   1.0816
1575       ergebni   18.787010   19.291093   Topic3  -5.8536   1.0810
1892        folgen   18.793743   19.298126   Topic3  -5.8532   1.0810
6123  wissenschaft   18.732033   19.236529   Topic3  -5.8565   1.0809

[120 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
96        2  0.988659      aktuel
126       1  0.983412         alt
157       3  0.988667      analys
254       3  0.976560      arbeit
440       3  0.998697  auswirkung
...     ...       ...         ...
6324      1  0.989861       das
6326      2  0.986189       der
6327      3  0.999703       die
6343      2  0.974366  öffentlich
6359      2  0.957229  österreich

[90 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2])